In [1]:
import pandas as pd
import numpy as np

# merge data from media
data = pd.DataFrame({'url':[], 'title':[], 'date':[], 'press':[], 'article':[]})
file_list = ['joongang', 'hk', 'SBS', 'KBS', 'MBC', 'chosun', 'hani', 'khan', 'donga', 'mk']
for name in file_list:
    data = pd.concat([data, pd.read_csv(name + '.csv', engine='python', encoding='utf8')], sort=True)

# drop duplicates and filter too short articles
data = data.dropna(subset=['article', 'date', 'press']).drop_duplicates()
data = data.drop_duplicates(subset=['article'], keep='first')
data = data[data['article'].str.len() > 50].reset_index(drop=True)

In [2]:
data['date'] = data['date'].str.replace(' ', '').str.replace('-', '.')

data['article'] = data['article'].str.replace('아무개', '모')
data['article'] = data['article'].str.replace('[가-힣]{2,3}\s{1,3}기자', '')
data['article'] = data['article'].str.replace(r'\s[김이박최정강조윤장임한오서신권황안송전홍류고문양손배조'
                                              '백허유남심노하A-Zㄱ-ㅎ]{1}\s{0,1}[모]{0,1}\s{0,'
                                              '1}\({1}.{1,10}\){1}\s{0,1}(?P<name>[양군씨])', ' 김\g<1> ')
data['article'] = data['article'].str.replace(r'\s[김이박최정강조윤장임한오서신권황안송전홍류고문양손배조'
                                              '백허유남심노하A-Zㄱ-ㅎ]{1}\s{0,1}\({1}.{1,'
                                              '10}\){1}\s{0,1}[모]{0,1}\s{0,1}(?P<name>[양군씨])', ' 김\g<1> ')
data['article'] = data['article'].str.replace(r'\s[김이박최정강조윤장임한오서신권황안송전홍류고문양손배조'
                                              '백허유남심노하A-Zㄱ-ㅎ]{1}\s{0,1}\s{0,1}[모]{0,1}\s{0,1}(?P<name>[양군씨])', ' 김\g<1> ')

data['article'] = data['article'].str.replace(r'\s[김이박최정강조윤장임한오서신권황안송전홍류고문양손배조'
                                              '백허유남심노하A-Zㄱ-ㅎ]\s{0,1}모\s{0,1}씨{1}', ' 김모씨 ')
data['article'] = data['article'].str.replace(r'\s[김이박최정강조윤장임한오서신권황안송전홍류고문양손배조'
                                              '백허유남심노하A-Zㄱ-ㅎ]\s{0,1}모\s{0,1} ', ' 김모씨 ')

data['article'] = data['article'].str.replace(r'[연합뉴스] <저작권자 ⓒ 1980-2018 ㈜연합뉴스. 무단 전재 재배포 금지.>', ' ')
data['article'] = data['article'].str.replace(r'연합뉴스 모바일', ' ')

data['article'] = data['article'].str.replace(' [김은진주윤양현이최박권배날신엄원곽'
                                        '연구변우조오송석민천전정홍강임장한유서'
                                        '선황고노공차안나함모방성]\s{0,1}모{0,1}'
                                        '\s{0,1}씨 ', ' 김모씨 ')
data['article'] = data['article'].str.replace(' [김은진주윤양현이최박권배날신엄원곽'
                                        '연구변우조오송석민천전정홍강임장한유서'
                                        '선황고노공차안나함모방성]\s{0,1}모{0,1}'
                                        '\s{0,1}양 ', ' 김모양 ')
data['article'] = data['article'].str.replace(' [김은진주윤양현이최박권배날신엄원곽'
                                        '연구변우조오송석민천전정홍강임장한유서'
                                        '선황고노차안나함모방성]\s{0,1}모{0,1}'
                                        '\s{0,1}군 ', ' 김모군 ')
data['pos'] = data['article']
data.to_csv('merged.csv', encoding='utf8', index=False)

In [38]:
data = pd.read_csv('merged.csv')

In [4]:
data = data.dropna(subset=['title', 'article']).reset_index(drop=True)

In [5]:
data = data[(data.title.str.contains('성범죄|성폭행|강간|성폭력|성추행|성희롱')) | 
         (data.article.str.count('성범죄|성폭행|강간|성폭력|성추행|성희롱|추행|미투') >= 1)].reset_index(drop=True)

In [6]:
from konlpy.tag import Komoran
import re

pos_tagger = Komoran()

def pos(doc): 
    
    stopwords = ['기자', '대한', '위해', '이후', '이나', '에서', '로부터', '면서', 
             '상대로', '하자', '중이', '로서', '대로', '이기', '구독',
            '구독','글방', '한수진', '사회자', '무단', '전재', '복제',
             '콘텐츠', '허브', '배포', '저작권', '베스트', '추천', '포토', '연합뉴스',
             '래빗', '뉴스래빗', '기사', '중앙',  '한경', '한경닷컴', '한겨레', '경향',
             '동아', '동아일보', '조선', '조선일보', '중앙일보', '매일경제', '매일',
             '뉴스', '앵커', '있다', '있는', '하는', 
             '입니', '있었', '아니', '없다', '이런', '않았', '하지', '하기', '하게', 
             '하다', '이어', '하려', '관한', '있어', '이를', '않고', '않은', 
             '어떤', '많은', '하며', '어떻', '있지', '있던', '아닌', '있으', '있고', 
             '있도', '하거', '하던', '하도', '그러', '겁니', '보여', '있거']
    doc = re.sub('|'.join(stopwords), '', doc)
    return ['/'.join(t) for t in pos_tagger.pos(doc) 
            if re.search('NNG|NNP|VV|VA|MM', t[1]) != None and re.search('[0-9A-Za-z]', t[0]) == None]

In [7]:
data['pos'] = [pos(i) for i in data['pos']]

In [38]:
data['pos'] = [' '.join(i) for i in data['pos']]

In [9]:
data.pos = data.pos.str.replace('/VV|/VA', '다/VV').str.replace('/[A-Z]{2,3}', '')

In [13]:
for i in data[data.pos.str.contains('김 모 군') & data.pos.str.contains('육군|병장|상병|군대|헌병대|군기|부대|김군기')].index:
    data.loc[i, 'pos'] = re.sub(data.loc[i, 'pos'], '김 모 군', '육군')

In [15]:
data['pos'] = data['pos'].str.replace('갈리아', '메갈리아')
data['pos'] = data['pos'].str.replace('드 루킹', '드루킹')
data['pos'] = data['pos'].str.replace('더불 민주', '더불어민주당')
data['pos'] = data['pos'].str.replace('와인 스티|와인 스타', '와인스타인')
data['pos'] = data['pos'].str.replace('스트로스 칸', '스트로스칸')
data['pos'] = data['pos'].str.replace('양 예원', '양예원')
data['pos'] = data['pos'].str.replace('승우 |여가 ', '')
data['pos'] = data['pos'].str.replace(' 여신 도 | 여신[도]{1,4} | 여신 ', ' 여신도 ')
data['pos'] = data['pos'].str.replace('피의자|용의자', '가해자')
data['pos'] = data['pos'].str.replace('\s{2,}', '\s')
data.pos = data.pos.str.replace(' 김 모양 ', ' 김양 ')
data.pos = data.pos.str.replace(' 김 모 군 ', ' 김군 ')
data.pos = data.pos.str.replace(' 김 모씨 | 김모씨 ', ' 김씨 ')
data.pos = data.pos.str.replace('[김은진주윤양현이최박권배신엄원곽'
                                        '연구변우조오송석민천전정홍강임장한유서'
                                        '황노차나방성]\s{0,1}모{1}'
                                        '\s{0,1}(?P<name>[양군씨])', '김\g<1>')
data.pos = data.pos.str.replace('[김은진주윤양현이최박권배신엄원곽'
                                        '연구변우조오송석민천전정홍강임장한유서'
                                        '황노차나방성]\s{0,1}모{0,1}'
                                        '\s{0,1}씨', '김씨')
data.pos = data.pos.str.replace(' 양 ', ' 김양 ')
data.pos = data.pos.str.replace(' [박최정강윤임한오서신권황안송홍류문손배백허유남심하]모 ', ' 김씨 ')

In [47]:
for i in data[data.pos.str.contains('김군') & data.pos.str.contains('육군|병장|상병|군대|헌병대|군기|부대|김군기')].index:
    data.loc[i, 'pos'] = re.sub(data.loc[i, 'pos'], '김군', '육군')

In [19]:
data.pos = data.pos.str.replace(' 곽 | 한 | 이 | 말 | 전 | 뒤 | 그 | 때 | 고 | 일 | 성 | 게 | 두 | 이 '
                                '| 하 | 록 | 후 | 안 | 앞 | 모 | 라 | 을 | 의 | 달 | 으 | 주 | 범 | 방 | 곳 | 다 |'
                                ' 면 | 당 | 직 | 날 | 미 | 위 | 반 | 도 | 부 | 약 | 단 | 끝 | 장 | 차 | 세 | 뒤 | 한 | 현 | 급 '
                                ' 사 | 실 | 뜻 | 시 | 요 | 올 | 인 | 새 | 옆 | 르 | 하다 | 되다 | 없다 | 가다 | 같다 |'
                                ' 그 | 사 | 급  ', ' ')

In [21]:
data.pos = data.pos.str.replace('저렇다|아무렇다|거리다 ', ' ')
data.pos = data.pos.str.replace(r'\s{2,}', ' ')
data.pos = data.pos.str.replace('모씨', '김씨')
data.pos = data.pos.str.replace('영이', '나영이')
data.pos[data[data.pos.str.contains(' 군 ') & data.pos.str.contains(' 김군 ')].index] = data.pos[
    data[data.pos.str.contains(' 군 ') & data.pos.str.contains(' 김군 ')].index].str.replace(' 군 ', ' 김군 ')

In [24]:
import nltk
from nltk import FreqDist

tokens = [j for i in data['pos'] for j in i.split(' ')]
text = nltk.Text(tokens)
freq = FreqDist(text)
sorted(freq.items(), key=lambda item: item[1], reverse=True)

[('김씨', 220860),
 ('받다', 138856),
 ('여성', 135783),
 ('경찰', 115121),
 ('사건', 112022),
 ('혐의', 107858),
 ('성폭행', 105661),
 ('늘다', 101602),
 ('밝히다', 85370),
 ('성폭력', 75016),
 ('피해자', 74898),
 ('성추행', 74450),
 ('보다', 73998),
 ('지나다', 72953),
 ('대하다', 72054),
 ('조사', 71138),
 ('범죄', 60888),
 ('문제', 56389),
 ('사실', 55117),
 ('피해', 53638),
 ('수사', 53437),
 ('사람', 52678),
 ('따르다', 51072),
 ('자신', 50371),
 ('의원', 45974),
 ('김양', 44947),
 ('서울', 44538),
 ('사회', 43092),
 ('관련', 42926),
 ('경우', 42128),
 ('가해자', 40548),
 ('성희롱', 40121),
 ('검찰', 39908),
 ('미국', 39596),
 ('지난해', 39258),
 ('학생', 37833),
 ('나오다', 37553),
 ('선고', 37521),
 ('처벌', 37228),
 ('주장', 37114),
 ('학교', 36987),
 ('성범죄', 36029),
 ('통하다', 35806),
 ('시간', 34300),
 ('교수', 33780),
 ('당하다', 33545),
 ('기소', 33366),
 ('아동', 32057),
 ('크다', 31837),
 ('알리다', 31825),
 ('법', 31449),
 ('정부', 31209),
 ('범행', 31130),
 ('다른', 31066),
 ('남성', 30928),
 ('오다', 30710),
 ('결과', 30585),
 ('최근', 30574),
 ('대통령', 30454),
 ('당시', 30443),
 ('집', 30309),
 

In [25]:
[i for i in sorted(freq.items(), key=lambda item: item[1], reverse=True) if len(i[0])==1]

[('법', 31449),
 ('집', 30309),
 ('말', 23581),
 ('김', 23149),
 ('술', 19073),
 ('박', 18178),
 ('딸', 15934),
 ('글', 14059),
 ('돈', 13681),
 ('속', 13608),
 ('팀', 12648),
 ('이', 12334),
 ('손', 11458),
 ('강', 11209),
 ('군', 10866),
 ('정', 10734),
 ('문', 10370),
 ('몸', 10327),
 ('한', 10294),
 ('여', 10234),
 ('배', 10046),
 ('길', 9684),
 ('첫', 9452),
 ('밤', 8548),
 ('형', 8232),
 ('윤', 7983),
 ('남', 7877),
 ('죄', 7872),
 ('입', 7408),
 ('총', 7317),
 ('그', 7144),
 ('책', 6870),
 ('몇', 6669),
 ('급', 6599),
 ('최', 6372),
 ('눈', 6337),
 ('달', 6315),
 ('전', 6293),
 ('밖', 6235),
 ('때', 5939),
 ('삶', 5876),
 ('답', 5828),
 ('뒤', 5810),
 ('힘', 5677),
 ('목', 5513),
 ('옷', 5490),
 ('일', 5475),
 ('각', 5250),
 ('발', 5087),
 ('물', 4931),
 ('병', 4929),
 ('귀', 4830),
 ('비', 4672),
 ('국', 4553),
 ('예', 4452),
 ('냐', 4398),
 ('로', 4322),
 ('지', 4249),
 ('관', 4164),
 ('신', 4077),
 ('피', 4069),
 ('홍', 4010),
 ('해', 3935),
 ('과', 3864),
 ('선', 3864),
 ('토', 3861),
 ('순', 3803),
 ('영', 3761),
 ('감', 3657),
 ('상', 3648),

날짜 단위로 잘라서 저장

In [26]:
data['date'] = [i[:7] for i in data['date']]

In [27]:
for i in data[data['date'].str.len() == 4]['date'].index:
    data.loc[i, 'date'] = data['date'][i] + '.01.01'
data = data[data['date'].str.len() >= 7].reset_index(drop=True)
data = data[data['date'].str.contains('[^\d.]') == False].reset_index(drop=True)

In [28]:
year = 2002
month = 1

In [29]:
import numpy as np
data['dindex'] = [np.nan for _ in range(0, len(data['url']))]
data['dindex'] = [12*(int(i.split('.')[0]) - year) + int(i.split('.')[1]) - month for i in data['date']]

In [30]:
data = data[data['date'] > '2002.01']

In [31]:
data['date'] = [i[:7] for i in data['date']]

In [32]:
data = data.dropna(subset=['article', 'date', 'press', 'pos'])

In [33]:
data['num'] = [len(i.split(' ')) for i in data['pos']]

In [34]:
data.to_csv('processed_2002.csv', encoding='utf8', index=False)

In [2]:
import sys
import csv
import pandas as pd

data = pd.read_csv('processed_2002.csv', encoding='utf8', engine='python').dropna(subset=['pos'])

In [51]:
from gensim.models.word2vec import Word2Vec

data['pos'] = [i.split(' ') for i in data['pos']]
model = Word2Vec(data['pos'], size=300, alpha=0.025, min_alpha=0.025,
                 window=5, workers=14, min_count=150)
model.save('standard.model')

In [39]:
data.to_csv('processed_2002.csv', encoding='utf8', index=False)